# World
Implements a world class which integrates an environment with one or more observables and several robots.

In [1]:
from Environment import Environment, PollutionModelEnvironment, EpidemicSpreadEnvironment
from InformationModel import InformationModel, ScalarFieldInformationModel_stored_observation
from Robot import Robot
from Policy import GoToLocationPolicy, FollowPathPolicy, RandomWaypointPolicy

import logging
# logging.getLogger().setLevel(logging.DEBUG)
# logging.getLogger().setLevel(logging.WARNING)
logging.getLogger().setLevel(logging.ERROR)

In [2]:
class World:
    def __init__(self, env, im):
        self.env = env
        self.im = im
        self.width, self.height = env.width, env.height
        self.time = 0
        self.robots = []
                
    def add_robot(self, robot):
        self.robots.append(robot)
        
    def enact_policy(self, delta_t = 1.0):
        """For all the agents in the environment, it calls 
        their policies to allow for the scheduling of the actions."""
        for robot in self.robots:
            robot.enact_policy(delta_t)
        
    def proceed(self, delta_t = 1.0):
        """Implements the passing of time"""
        for robot in self.robots:
            robot.proceed(delta_t)
        self.env.proceed(delta_t)
        self.im.proceed(delta_t)
        self.time = self.time + delta_t

In [ ]:
def generate_environment():
    """Function to generate the environment model"""
    env = PollutionModelEnvironment("water", 10, 10, seed = 1)
    env.evolve_speed = 1
    env.p_pollution = 0.1
    for i in range(100):
        env.proceed(1.0)
    # setting it to unchanging
    env.evolve_speed = 0
    env.p_pollution = 0.0
    return env

In [ ]:
def generate_information_model(settings, env):
    """Generates an information model appropriate to the environment and the estimation 
    type described by the settings"""
    im = ScalarFieldInformationModel_stored_observation("sample", 
                        env.width, env.height, 
                        estimation_type=settings["estimation_type"])
    return im

In [ ]:
def generate_robots(settings, world):
    """Generate the robots and their policies according to the settings. 
    Current version generate n robots with random waypoint behavior"""
    count = settings["count"]
    for ib in range(count):
        robot = Robot(f"Robot-{i}", 0, 0, 0, env=world.env, im=world.im)
        robot.policy = RandomWaypointPolicy(None, robot, 1, [0,0], [9, 9])
        world.add_robot(robot)

In [ ]:
def run_experiment(genenv, genim, genrob):
    """Generates the environment, the model and the robots"""
    env = genenv()
    im = genim(env)
    world = World(env, im)
    genrob(world)
    record = []
    for t in range(100):
        world.enact_policy(1)
        world.proceed(1)
        ## this is where we get some kind of metric
        record.append(im.score(env))
    return record

In [4]:
# environment model
env = PollutionModelEnvironment("water", 10, 10)
env.evolve_speed = 1
env.p_pollution = 0.1
for i in range(100):
    env.proceed(1.0)
# setting it to unchanging
env.evolve_speed = 0
env.p_pollution = 0.0
im = ScalarFieldInformationModel_stored_observation("sample", env.width, env.height, 
                                estimation_type="gaussian-process")
im.proceed(0)

world = World(env, im)
robi = Robot("Robi", 2, 3, 0, env=env, im=im)
# robot.policy = GoToLocationPolicy(None, robot, 9, 9, 0.2)
world.add_robot(robi)
robur = Robot("Robur", 4, 8, 0, env=env, im=im)
robur.policy = RandomWaypointPolicy(None, robur, 1, [0,0], [9, 9])
world.add_robot(robur)
r2d3 = Robot("R2D3", 5, 9, 0, env=env, im=im)
r2d3.policy = FollowPathPolicy(None, r2d3, 1, [[0,0], [5, 5], [9,0]], repeat = True)
world.add_robot(r2d3)

In [5]:
for i in range(10):
    world.enact_policy(1)
    world.proceed(1)
print(robur.location_history)

C:\Users\lboloni\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 10. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
C:\Users\lboloni\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:411: ConvergenceWarning: The optimal value found for dimension 1 of parameter k1__length_scale is close to the specified upper bound 10. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
C:\Users\lboloni\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "

deque([[2.4459010232187355, 4.350853042225939, 0], [2.6463008801033774, 3.3711388505554214, 0], [2.8467007369880193, 2.391424658884904, 0], [3.047100593872661, 1.411710467214387, 0], [3.141125108917501, 2.0617903437594904, 0], [3.2842709240979175, 3.0514919531329086, 0], [3.4274167392783337, 4.041193562506327, 0], [3.5705625544587503, 5.030895171879745, 0], [3.713708369639167, 6.020596781253164, 0], [3.8568541848195834, 7.010298390626582, 0]])


C:\Users\lboloni\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 10. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
C:\Users\lboloni\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:411: ConvergenceWarning: The optimal value found for dimension 1 of parameter k1__length_scale is close to the specified upper bound 10. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
C:\Users\lboloni\anaconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__length_scale is close to the specified upper bound 10. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
C

In [7]:
[x[0] for x in robur.location_history]

[2.4459010232187355,
 2.6463008801033774,
 2.8467007369880193,
 3.047100593872661,
 3.141125108917501,
 3.2842709240979175,
 3.4274167392783337,
 3.5705625544587503,
 3.713708369639167,
 3.8568541848195834]